In [6]:
import polars as pl
import pandas as pd
import os

In [15]:
folder_data_processed = '../data/processed'

def get_data_country(df_name,country):
    df_data_country = pd.read_csv(os.path.join(folder_data_processed,country,df_name+'.csv'))
    return(df_data_country)

country='spanje'
df_m_country = get_data_country('df_m_country',country)
df_m_country

,m,tmin_med,tavg_med,tmax_med,wspd_med,rain_sum,rain_pct,prcp_sum,country,place
0,1,7.5,11.3,17.3,14.8,4.0,0.13,3.0,spanje,alicante
1,2,7.9,12.5,18.5,14.4,4.0,0.12,2.0,spanje,alicante
2,3,9.9,14.4,19.5,15.7,9.0,0.29,38.0,spanje,alicante
3,4,12.1,16.6,21.3,13.6,8.0,0.27,31.0,spanje,alicante
4,5,15.2,20.0,24.8,12.5,5.0,0.17,23.0,spanje,alicante
...,...,...,...,...,...,...,...,...,...,...
391,8,22.9,27.3,32.5,11.1,2.0,0.08,1.0,spanje,villajoyosa
392,9,20.2,24.1,29.5,11.5,7.0,0.25,24.0,spanje,villajoyosa
393,10,15.5,20.4,25.6,10.8,5.0,0.16,28.0,spanje,villajoyosa
394,11,12.0,15.8,22.0,13.7,6.0,0.18,12.0,spanje,villajoyosa


In [14]:
from openpyxl import load_workbook

def xlsx_sheet_to_df(filename:str,sheetname:str) -> pd.DataFrame:
    wb = load_workbook(filename)
    ws = wb[sheetname]
    data = list(ws.values)
    headers=next(iter(data))
    df_out = pd.DataFrame(data[1:],columns=headers)
    return(df_out)

def create_dict_destinations(df_destinations):
    cols = list(df_destinations.columns)
    dict_destinations = {}
    for i,v in df_destinations.iterrows():
        country = v['Country'].lower().replace(' ','-')
        place = v['Place'].lower().replace(' ','-')
        lat = v['Lat']
        if country not in dict_destinations:
            dict_destinations[country]={}
        dict_destinations[country][place] = {col:v[col] for col in cols}
        
    return(dict_destinations)


df_destinations = xlsx_sheet_to_df(filename=r'../data/destinations.xlsx',sheetname='Destinations')
dict_destinations = create_dict_destinations(df_destinations)
dict_destinations

{'egypte': {'cairo': {'Country': 'Egypte',
   'Place': 'Cairo',
   'Region': None,
   'Type': 'stad',
   'Lat': 30.0443879,
   'Lon': 31.2357257},
  'hurghada': {'Country': 'Egypte',
   'Place': 'Hurghada',
   'Region': 'Rode Zee',
   'Type': 'zon',
   'Lat': 27.222556,
   'Lon': 33.8307062},
  'luxor': {'Country': 'Egypte',
   'Place': 'Luxor',
   'Region': None,
   'Type': 'zon',
   'Lat': 25.702096,
   'Lon': 32.647186},
  'marsa-alam': {'Country': 'Egypte',
   'Place': 'Marsa Alam',
   'Region': 'Rode Zee',
   'Type': 'zon',
   'Lat': 25.069992,
   'Lon': 34.893817},
  'sharm-el-sheikh': {'Country': 'Egypte',
   'Place': 'Sharm el Sheikh',
   'Region': 'Rode Zee',
   'Type': 'zon',
   'Lat': 27.8644422,
   'Lon': 34.295447}},
 'griekenland': {'agistri': {'Country': 'Griekenland',
   'Place': 'Agistri',
   'Region': None,
   'Type': 'zon',
   'Lat': 37.6937039,
   'Lon': 23.35005226086773},
  'athene': {'Country': 'Griekenland',
   'Place': 'Athene',
   'Region': None,
   'Type': 's

In [27]:
def create_df_map(df_data_country,dict_destinations,cols_keep=['m','tavg_med']):
    df_out = df_data_country.copy()
    def get_v(row,v='Lat'):
        return dict_destinations[row['country']][row['place']][v]
    df_out['Lat'] = df_out.apply(get_v,args=('Lat',),axis=1)
    df_out['Lon'] = df_out.apply(get_v,args=('Lon',),axis=1)
    df_out['Place'] = df_out.apply(get_v,args=('Place',),axis=1)
    df_out = df_out[['Place','Lat','Lon']+cols_keep]
    return(df_out)

create_df_map(df_m_country,dict_destinations)

,Place,Lat,Lon,m,tavg_med
0,Alicante,38.343637,-0.488171,1,11.3
1,Alicante,38.343637,-0.488171,2,12.5
2,Alicante,38.343637,-0.488171,3,14.4
3,Alicante,38.343637,-0.488171,4,16.6
4,Alicante,38.343637,-0.488171,5,20.0
...,...,...,...,...,...
391,Villajoyosa,38.510078,-0.229620,8,27.3
392,Villajoyosa,38.510078,-0.229620,9,24.1
393,Villajoyosa,38.510078,-0.229620,10,20.4
394,Villajoyosa,38.510078,-0.229620,11,15.8


In [1]:
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Hourly
from typing import List, Dict, Tuple, Optional
import pandas as pd
from openpyxl import load_workbook
from geopy.geocoders import Nominatim
import numpy as np
import os
import math
import polars as pl
import polars.selectors as cs


## -------- Input data -------- 


def xlsx_sheet_to_df(filename:str,sheetname:str) -> pd.DataFrame:
    wb = load_workbook(filename)
    ws = wb[sheetname]
    data = list(ws.values)
    headers=next(iter(data))
    df_out = pd.DataFrame(data[1:],columns=headers)
    return(df_out)


def xlsx_add_latlon(filename:str,sheetname:str):
    df_xlsx = xlsx_sheet_to_df(filename,sheetname)
    geolocator = Nominatim(user_agent='myapplication')
    boolean_overwrite = False
    for i,v in df_xlsx.iterrows():
        lat = v['Lat']
        # print(lat)
        # print(np.isnan(lat) or lat is None)
        if np.isnan(lat) or lat is None:
            boolean_overwrite = True
            dest = f"{v['Place']}, {v['Country']}"
            print(dest)
            loc = geolocator.geocode(dest)
            df_xlsx.loc[i,('Lat')]=float(loc.raw['lat'])
            df_xlsx.loc[i,('Lon')]=float(loc.raw['lon'])
    if boolean_overwrite:
        print(f'Added latlon to {filename}')
        df_xlsx.to_excel(filename,sheet_name=sheetname,index=False)
    return(df_xlsx)


def create_dict_destinations(df_destinations):
    cols = list(df_destinations.columns)
    dict_destinations = {}
    for i,v in df_destinations.iterrows():
        country = v['Country'].lower().replace(' ','-')
        place = v['Place'].lower().replace(' ','-')
        lat = v['Lat']
        if country not in dict_destinations:
            dict_destinations[country]={}
        dict_destinations[country][place] = {col:v[col] for col in cols}
        
        # if lat is None or np.isnan(lat):
        #     dict_latlon = add_latlon(country=v['Country'],place=v['Place'])
        #     dict_destinations[country][place].update(dict_latlon)
    return(dict_destinations)


## -------- Download data (helper functions) -------- 


def get_weather(lat,lon,start,end,frequency='daily'):
    if frequency=='daily':
        data = Daily(Point(lat,lon),start,end)
    elif frequency=='hourly':
        data = Hourly(Point(lat,lon),start,end)
    data = data.fetch()
    data = data[data.index<(datetime.now()-timedelta(days=1))]
    return data


def download_add_daily(country,place,dict_destinations,last_n_years=7):
    dict_cp = dict_destinations[country][place]
    lat,lon = dict_cp['Lat'],dict_cp['Lon']
    days = math.ceil(last_n_years*365.25)
    start = (datetime.today()-timedelta(days=days))
    end = datetime.today()
    data_raw = get_weather(lat=lat,lon=lon,start=start,end=end,frequency='daily')
    if data_raw.shape[0]>0:
        df_cp_d = pl.from_pandas(data_raw,include_index=True).select(
            ['time','tavg','tmin','tmax','prcp','wspd','pres']
        ).cast(
            {'time':pl.Date,
            'tavg':pl.Float64,'tmin':pl.Float64,'tmax':pl.Float64,
            'prcp':pl.Float64,'wspd':pl.Float64,'pres':pl.Float64}
        ).filter(
            (pl.col('time').max()-pl.col('time')).dt.total_days()<=(365.25*last_n_years)
        ).with_columns(
            # pl.col('time').dt.strftime('%m-%d').alias('md'),
            pl.col('time').dt.year().alias('y'),
            # pl.col('time').dt.strftime('%m').alias('m_'),
            pl.col('time').dt.month().alias('m'),
            pl.when(pl.col("time").dt.day() > 20)
                .then(3)
                .when(pl.col("time").dt.day() > 10)
                .then(2)
                .otherwise(1)
                .alias("m3"),
            pl.col('time').dt.day().alias('d'),
            pl.when(pl.col('prcp')==0).then(0).when(pl.col('prcp')>0).then(1).otherwise(None).alias('rain')
        ).cast({'m3':pl.Int8,'rain':pl.Int8})
        return(df_cp_d)


def download_add_hourly(country,place,dict_destinations,last_n_years=7):
    dict_cp = dict_destinations[country][place]
    lat,lon = dict_cp['Lat'],dict_cp['Lon']
    days = math.ceil(last_n_years*365.25)
    start = (datetime.today()-timedelta(days=days))
    end = datetime.today()
    data_raw = get_weather(lat=lat,lon=lon,start=start,end=end,frequency='hourly')
    if data_raw.shape[0]>0:
        df_cp_h = pl.from_pandas(data_raw,include_index=True).select(
            ['time','temp','rhum','prcp','wspd','pres','coco']
        ).cast(
            {'time':pl.Datetime,
            'temp':pl.Float64,'rhum':pl.Float64,
            'prcp':pl.Float64,'wspd':pl.Float64,'pres':pl.Float64,
            'coco':pl.Float64
            }
        ).with_columns(
            pl.col('time').dt.year().alias('y'),
            pl.col('time').dt.month().alias('m'),
            pl.col('time').dt.day().alias('d'),
            pl.col('time').dt.hour().alias('h'),
            pl.when(pl.col("time").dt.day() > 20)
                .then(3)
                .when(pl.col("time").dt.day() > 10)
                .then(2)
                .otherwise(1)
                .alias("m3"),
            pl.when(pl.col('prcp')==0).then(0).when(pl.col('prcp')>0).then(1).otherwise(None).alias('rain')
        ).cast({'m3':pl.Int8,'rain':pl.Int8})
        return (df_cp_h)


## -------- Preprocess and save data -------- 


def dictdfs_write_csv(dict_dfs,country,place=None,folder_data_processed='./data/processed'):
    dir_csv = os.path.join(folder_data_processed,'' if (country is None) else country,'' if (place is None) else place )
    for df in dict_dfs:
        os.makedirs(dir_csv,exist_ok=True)
        fp_csv = os.path.join(dir_csv,df+'.csv')
        dict_dfs[df].write_csv(fp_csv,separator=',')


def preprocess_daily(df_cp_d):
    """Create dict with multiple aggregate df's: mm3d, mm3, m
    All averages (except rain) are the averages of the upper and lower bound, defined as 90% quantile
    rain_pct, rain_sum is taken over all days (i.e. not deleting extreme points)
    """
    # df_cp_d = load_filter_add_daily(country=country,place=place,last_n_years=last_n_years,folder_data_raw=folder_data_raw)
    # df_cp_d = download_add_daily(country,place,dict_destinations,last_n_years=last_n_years)
    cols_d = ['tmin','tavg','tmax','prcp','wspd']
    df_ub_d = df_cp_d.group_by(['m','m3','d']).agg(
        pl.col(c).quantile(0.9,'lower').alias(str(c)+'_ub') for c in cols_d
    )
    df_lb_d = df_cp_d.group_by(['m','m3','d']).agg(
        pl.col(c).quantile(0.1,'higher').alias(str(c)+'_lb') for c in cols_d
    )
    df_rain = df_cp_d.group_by(['m','m3','d']).agg(
        pl.col('rain').mean().round(2).alias('rain_pct')
    )    
    df_mm3d = df_lb_d.join(
        df_ub_d,on=['m','m3','d'],how='full', coalesce=True
    ).join(
        df_rain,on=['m','m3','d'],how='full', coalesce=True
    ).with_columns(
        ((pl.col(str(c)+'_ub')+pl.col(str(c)+'_lb'))/2).round(1).alias(str(c)+'_avg') for c in cols_d
        ).select(
            cs.by_dtype([pl.Int8]),cs.matches('_avg$'),cs.matches('_pct$')
        ).sort(['m','d'])

    df_m = df_mm3d.group_by(['m']).agg(
        cs.matches('^tmin|^tmax|^tavg|^wspd').median().round(1).name.map(lambda c:c.replace('_avg','_med')),
        
        cs.matches('rain_pct').sum().round(0).alias('rain_sum'),
        cs.matches('rain_pct').mean().round(2).alias('rain_pct'),
        cs.matches('prcp').sum().round(0).name.map(lambda c:c.replace('_avg','_sum'))
    ).sort(['m'])

    df_mm3 = df_mm3d.group_by(['m','m3']).agg(
        cs.matches('^tmin|^tmax|^tavg|^wspd').median().round(1).name.map(lambda c:c.replace('_avg','_med')),
        cs.matches('rain_pct').sum().round(0).alias('rain_sum'),
        cs.matches('rain_pct').mean().round(2).alias('rain_pct'),
        cs.matches('prcp').sum().round(0).name.map(lambda c:c.replace('_avg','_sum'))
    ).sort(['m','m3'])
    return({'df_mm3d':df_mm3d,
            'df_mm3':df_mm3,
            'df_m':df_m
           })


def preprocess_hourly(df_cp_h):
    # df_cp_h = download_add_hourly(country,place,dict_destinations,last_n_years=last_n_years)
    cols_h = ['temp','rhum','prcp','wspd'] # and coco
    df_ub_h = df_cp_h.group_by(['m','m3','d','h']).agg(
        pl.col(c).quantile(0.9,'lower').alias(str(c)+'_ub') for c in cols_h
    )
    df_lb_h = df_cp_h.group_by(['m','m3','d','h']).agg(
        pl.col(c).quantile(0.1,'higher').alias(str(c)+'_lb') for c in cols_h
    )
    df_rain = df_cp_h.group_by(['m','m3','d','h']).agg(
        pl.col('rain').mean().round(2).alias('rain_pct')
    )  
    
    # df_mm3dh used for other calcs, not part of output:
    df_mm3dh = df_lb_h.join(
        df_ub_h,on=['m','m3','d','h'],how='full',coalesce=True
    ).join(
        df_rain,on=['m','m3','d','h'],how='full', coalesce=True
    ).with_columns(
        ((pl.col(str(c)+'_ub')+pl.col(str(c)+'_lb'))/2).round(1).alias(str(c)+'_avg') for c in cols_h
        ).select(
        cs.by_dtype([pl.Int8]),pl.col('temp_lb'),pl.col('temp_ub'),cs.matches('_avg$'),cs.matches('_pct$')
    )
    df_mm3h = df_mm3dh.group_by(['m','m3','h']).agg(
        cs.matches('temp|rhum|wspd').median().round(1).name.map(lambda c:c.replace('_avg','_med')),
        cs.matches('rain_pct').mean().round(2).alias('rain_pct'),
    ).sort(['m','m3','h'])
    df_mh = df_mm3dh.group_by(['m','h']).agg(
        cs.matches('temp|rhum|wspd').median().round(1).name.map(lambda c:c.replace('_avg','_med')),
        cs.matches('rain_pct').mean().round(2).alias('rain_pct')
    ).sort(['m','h'])

    # coco calculations, df_mm3dhc used for other calcs
    df_mm3dhc = df_cp_h.group_by(
        ['m','m3','d','h','coco']
    ).len().cast({'len':pl.Int8}).filter(
        pl.col('coco').is_not_null()
    ).filter(
        pl.col('coco')>0
    ).pivot(
        'coco',index=['m','m3','d','h'],values='len'
    ).fill_null(0).unpivot(
        cs.exclude('m','m3','d','h'),index=['m','m3','d','h']
    ).cast({'variable':pl.Float64}).with_columns(
        (pl.col('value')/pl.col('value').sum().over(['m','m3','d','h'])).round(3)
    ).rename({'variable':'coco','value':'coco_pct'})

    # df_mhc = df_mm3dhc.group_by(['m','h','coco']).agg(pl.col('coco_pct').mean().round(3))
    df_mc = df_mm3dhc.group_by(['m','coco']).agg(pl.col('coco_pct').mean().round(3))
    df_mm3c = df_mm3dhc.group_by(['m','m3','coco']).agg(pl.col('coco_pct').mean().round(3))
    
    return({
        'df_mc':df_mc,
        'df_mm3c':df_mm3c,
        'df_mm3h':df_mm3h,
        'df_mh':df_mh
    })


# def download_add_daily(country,place,dict_destinations,last_n_years=7):
#     dict_cp = dict_destinations[country][place]
#     lat,lon = dict_cp['Lat'],dict_cp['Lon']
#     days = math.ceil(last_n_years*365.25)
#     start = (datetime.today()-timedelta(days=days))
#     end = datetime.today()
#     data_raw = get_weather(lat=lat,lon=lon,start=start,end=end,frequency='daily')
#     df_cp_d = pl.from_pandas(data_raw,include_index=True).select(
#         ['time','tavg','tmin','tmax','prcp','wspd','pres']
#     ).cast(
#         {'time':pl.Date,
#          'tavg':pl.Float64,'tmin':pl.Float64,'tmax':pl.Float64,
#          'prcp':pl.Float64,'wspd':pl.Float64,'pres':pl.Float64}
#     ).filter(
#         (pl.col('time').max()-pl.col('time')).dt.total_days()<=(365.25*last_n_years)
#     ).with_columns(
#         # pl.col('time').dt.strftime('%m-%d').alias('md'),
#         pl.col('time').dt.year().alias('y'),
#         # pl.col('time').dt.strftime('%m').alias('m_'),
#         pl.col('time').dt.month().alias('m'),
#         pl.when(pl.col("time").dt.day() > 20)
#             .then(3)
#             .when(pl.col("time").dt.day() > 10)
#             .then(2)
#             .otherwise(1)
#             .alias("m3"),
#         pl.col('time').dt.day().alias('d'),
#         pl.when(pl.col('prcp')==0).then(0).when(pl.col('prcp')>0).then(1).otherwise(None).alias('rain')
#     ).cast({'m3':pl.Int8,'rain':pl.Int8})
#     return(df_cp_d)



# def download_add_hourly(country,place,dict_destinations,last_n_years=7):
#     """Download hourly data, and add columns. Output is df, no csv is saved"""
#     dict_cp = dict_destinations[country][place]
#     lat,lon = dict_cp['Lat'],dict_cp['Lon']
#     days = math.ceil(last_n_years*365.25)
#     start = (datetime.today()-timedelta(days=days))
#     end = datetime.today()
#     data_raw = get_weather(lat=lat,lon=lon,start=start,end=end,frequency='hourly')
#     df_cp_h = pl.from_pandas(data_raw,include_index=True).select(
#         ['time','temp','rhum','prcp','wspd','pres','coco']
#     ).cast(
#         {'time':pl.Datetime,
#          'temp':pl.Float64,'rhum':pl.Float64,
#          'prcp':pl.Float64,'wspd':pl.Float64,'pres':pl.Float64,
#          'coco':pl.Float64
#         }
#     ).with_columns(
#         pl.col('time').dt.year().alias('y'),
#         pl.col('time').dt.month().alias('m'),
#         pl.col('time').dt.day().alias('d'),
#         pl.col('time').dt.hour().alias('h'),
#         pl.when(pl.col("time").dt.day() > 20)
#             .then(3)
#             .when(pl.col("time").dt.day() > 10)
#             .then(2)
#             .otherwise(1)
#             .alias("m3"),
#         pl.when(pl.col('prcp')==0).then(0).when(pl.col('prcp')>0).then(1).otherwise(None).alias('rain')
#     ).cast({'m3':pl.Int8,'rain':pl.Int8})
#     return (df_cp_h)


# # LEGACY:
# def get_weather_from_dict(start,end,dict_destinations,folder_data,frequency='daily',boolean_latest=True):
#     """Update all raw csv files for weather_daily and weather_hourly"""
#     for country in dict_destinations:
#         for place in dict_destinations[country]:
#             # Folder and file management:
#             dir = os.path.join(folder_data,country,place)
#             os.makedirs(dir,exist_ok=True)
#             fp = os.path.join(dir,f'weather_{frequency}.csv')
#             cp = dict_destinations[country][place]
#             lat,lon = cp['Lat'],cp['Lon']
#             boolean_download = (lat is not None and not np.isnan(lat)) and (lon is not None and not np.isnan(lon))
#             boolean_file = os.path.isfile(fp)
#             # boolean_download = boolean_download and place=='luxor' # TO DO, DELETE
#             if boolean_download:
#                 if boolean_file:
#                     # print(fp)
#                     df_file = pd.read_csv(fp,index_col='time')
#                     df_file.index = pd.to_datetime(df_file.index)
#                     # df_file.index.freq = 'd' if frequency=='daily' else 'h'
#                     if boolean_latest:
#                         start_file = max(df_file.index)-timedelta(days=1)
#                         end_file = datetime.today()
#                         df_download = get_weather(lat=lat,lon=lon,start=start_file,end=end_file,frequency=frequency)
#                     else:
#                         df_download = get_weather(lat=lat,lon=lon,start=start,end=end,frequency=frequency)
#                     df_merge = pd.concat([df_file,df_download])
#                     df_merge = df_merge[~df_merge.index.duplicated()]
#                     df_merge = df_merge.sort_index()
#                     df_merge.to_csv(fp,index=True)
#                 else:
#                     df_download = get_weather(lat=lat,lon=lon,start=start,end=end,frequency=frequency)
#                     if df_download.size>0:
#                         df_download.to_csv(fp,index=True)
#                     else:
#                         print(f'get_weather_from_dict(): empty data for {frequency} {country}, {place}')



# # def add_latlon(country,place):
# #     """Output: {'lat':0, 'lon':0}"""
# #     geolocator = Nominatim(user_agent='myapplication')
# #     loc = geolocator.geocode(f'{place}, {country}')
# #     dict_latlon = {'Lat':float(loc.raw['lat']),'Lon':float(loc.raw['lon'])}
# #     return dict_latlon





# # def get_lat_lon(dict_destinations:dict):
# #     geolocator = Nominatim(user_agent='myapplication')
# #     for dest in dict_destinations:
# #         if dict_destinations[dest]['Lat'] is None:
# #             print(f'get_lat_lon() for {dest}')
# #             loc = geolocator.geocode(dest)
# #             dict_destinations[dest]['Lat']=float(loc.raw['lat'])
# #             dict_destinations[dest]['Lon']=float(loc.raw['lon'])
# #     return(dict_destinations)



# # def get_lat_lon(list_destinations: List[str],dict_destinations:Dict[str, Tuple[float, float]]) -> Dict[str, Tuple[float, float]]:
# #     """

# #     Parameters
# #     ----------
# #     list_destinations: List[str] :
        
# #     dict_destinations:Dict[str,Tuple[float,float]] :
        

# #     Returns
# #     -------
# #     Dictionary with keys like 'Egypte, Hurghada' and values latitude 'lat' and longitude 'lon' as strings
# #     """
# #     for dest in list_destinations:
# #         if dest not in dict_destinations:
# #             print(f'get_lat_lon() for {dest}')
# #             loc = geolocator.geocode(dest)
# #             dict_destinations[dest]={
# #                 'lat':float(loc.raw['lat']),
# #                 'lon':float(loc.raw['lon']),
# #             }
# #     return dict_destinations


# # ## -------- Download data -------- 


# # def get_weather(destination: str,start: datetime,end: datetime,dict_destinations: Dict[str, Tuple[float, float]],frequency:str = 'daily',
# #                 cols_keep: Optional[List[str]]=None) -> pd.DataFrame:
# #     """

# #     Parameters
# #     ----------
# #     destination: str :
        
# #     start: datetime :
        
# #     end: datetime :
        
# #     dict_destinations: Dict[str,Tuple[float,float]] :
        
# #     frequency:str :
# #          (Default value = 'daily')
# #     cols_keep: Optional[List[str]] :
# #          (Default value = None)

# #     Returns
# #     -------

# #     """
# #         point = Point(dict_destinations[destination]['lat'],dict_destinations[destination]['lon'])
# #         if frequency=='daily':
# #               data = Daily(point, start, end)
# #         else:
# #               data = Hourly(point, start, end,model=True)
# #               data = data.fetch()
# #               data = data[data.index<(datetime.now()-timedelta(days=1))]
# #         if cols_keep is not None:
# #               data = data[[col for col in data.columns if col in cols_keep]]
        
# #         data[data.select_dtypes(include=['float64']).columns] = data.select_dtypes(include=['float64']).round(1)
# #         # print(destination)
# #         return data

In [2]:
df_destinations = xlsx_add_latlon(filename=r'../data/destinations.xlsx',sheetname='Destinations')
dict_destinations = create_dict_destinations(df_destinations)

# Update raw data:
# start = datetime(2018,1,1)
# end = datetime.today()
folder_data_processed = './data/processed'
last_n_years = 7

In [4]:
for country in dict_destinations:
    for place in dict_destinations[country]:
        print(f'{country}, {place}')
        df_cp_h = download_add_hourly(country,place,dict_destinations,last_n_years=last_n_years)

egypte, hurghada
egypte, marsa-alam
egypte, sharm-el-sheikh
egypte, cairo


egypte, luxor
italië, rome
nederland, ede
spanje, barcelona


spanje, palma-de-mallorca


spanje, malaga
turkije, alanya
turkije, antalya
turkije, bodrum
turkije, marmaris
